## Fake News Corpus
> https://huggingface.co/datasets/andyP/fake_news_en_opensources
- Satire
- Reliable news

> https://github.com/leereak/propaganda-detection
- Propaganda

> https://huggingface.co/datasets/liar
- Hoax

In [1]:
import csv
import pandas as pd
import numpy as np

from imblearn.under_sampling import RandomUnderSampler

In [2]:
csv.field_size_limit(500000)

131072

In [3]:
CHUNK_SIZE = 10 ** 6

In [9]:
df = pd.read_csv("opensources_fakenewscorpus_modified.csv", header=None)

df.iloc[:, 0].value_counts()

0
1    15000
2    15000
3    15000
4    15000
Name: count, dtype: int64

In [12]:
with pd.read_csv('opensources_fake_news_cleaned.csv', chunksize=CHUNK_SIZE, index_col="id") as reader:
    for chunk in reader:
        type_content = chunk[['type', 'content']]
        filtered = type_content[(type_content.type == 'satire') | (type_content.type == 'fake') | (type_content.type == 'bias') | (type_content.type == 'reliable')]
        filtered['type'] = filtered['type'].map({'satire': 1, 'fake': 2, 'bias': 3, 'reliable': 4})
        filtered.to_csv('fakenewscorpus.csv', mode='a', index=False, header=False)

C:\Users\Hong\AppData\Local\Temp\ipykernel_11272\2384829192.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['type'] = filtered['type'].map({'satire': 1, 'fake': 2, 'bias': 3, 'reliable': 4})
C:\Users\Hong\AppData\Local\Temp\ipykernel_11272\2384829192.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['type'] = filtered['type'].map({'satire': 1, 'fake': 2, 'bias': 3, 'reliable': 4})
C:\Users\Hong\AppData\Local\Temp\ipykernel_11272\2384829192.py:5: SettingWithCopyWarning: 
A value is tr

In [5]:
df = pd.read_csv('fakenewscorpus.csv', header=None, index_col=False)
# y = np.empty(0)
# X = np.empty(0)

# # Iterate over chunks
# for chunk in pd.read_csv('fakenewscorpus.csv', header=None, chunksize=CHUNK_SIZE):
#     y_chunk = chunk.iloc[:, 0].values  
#     X_chunk = chunk.iloc[:, 1].values  
    
#     y = np.concatenate((y, y_chunk))
#     X = np.concatenate((X, X_chunk))

: 

In [6]:
X = df.iloc[:, 1]
y = df.iloc[:, 0]

print(X.head())
print(y.value_counts())

0    FAIR USE NOTICE. Many of the stories on this s...
1    Abigail Ratchford‘s Instagram page is a force ...
2    Nothing says “I love you” like a sweat-stained...
3    Ryan Kovacik, a VCU premed student and certifi...
4    Quarters, beer pong, boat races, and flip cup ...
Name: 1, dtype: object
0
4    1807323
3     769874
2     762178
1      84735
Name: count, dtype: int64


In [7]:
rus = RandomUnderSampler(sampling_strategy={1: 15000, 2: 15000, 3: 15000, 4: 15000})
X_undersampled, y_undersampled  = rus.fit_resample(X.to_frame(), y.to_frame())

In [8]:
print(X_undersampled.head())
print(y_undersampled.value_counts())

                                                         1
514951   The phoney war was over today with Prime Minis...
3407701  The possibility that we earthlings are not tru...
3167716  With only a few days left for the official clo...
2321332  ‘Rules Are Rules,’ Say Those With Deeply Ingra...
3047105  MONTREAL — Canadian indie-rock icons Arcade Fi...
0
1    15000
2    15000
3    15000
4    15000
Name: count, dtype: int64


In [9]:
df_undersampled = pd.concat([y_undersampled, X_undersampled], axis=1)

In [10]:
X_undersampled = df_undersampled.iloc[:, 1]
y_undersampled = df_undersampled.iloc[:, 0]

print(X_undersampled.head())
print(y_undersampled.value_counts())

514951     The phoney war was over today with Prime Minis...
3407701    The possibility that we earthlings are not tru...
3167716    With only a few days left for the official clo...
2321332    ‘Rules Are Rules,’ Say Those With Deeply Ingra...
3047105    MONTREAL — Canadian indie-rock icons Arcade Fi...
Name: 1, dtype: object
0
1    15000
2    15000
3    15000
4    15000
Name: count, dtype: int64


In [11]:
df_undersampled.to_csv('opensources_fakenewscorpus_balancedtest.csv', index=False, header=False)